# Introduction

QA plots for the INTT detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for newer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_ana/ana.471`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

In [10]:

qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))


`qa_file_name_new` env not set. use the default `qa_file_name_new=prdf_reconstruction53877_qa.root`

In [11]:
# qa_file_new = ROOT.TFile.Open(qa_file_name_new);

# assert qa_file_new.IsOpen()
# qa_file_new.ls()
display(Markdown(f"Openning QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

Openning QA file at `prdf_reconstruction53877_qa.root`

22920909737984

In [12]:
%%cpp

if (qa_file_new == nullptr) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}
// list inputs histograms if needed
 //qa_file_new ->ls();

In [13]:
%%cpp

// Function to compute cluster averages
TH1* ClusterAverage(TH2* hist, std::string type) {
    int nX = hist->GetNbinsX();
    int nY = hist->GetNbinsY();

    auto graph = new TH1F(Form("avg_%s", type.c_str()), "", nX, hist->GetXaxis()->GetXmin(), hist->GetXaxis()->GetXmax());

    for (int i = 1; i <= nX; ++i) {
        double sum_y = 0;
        double totalEntries = 0;
        for (int j = 1; j <= nY; ++j) {
            double value = hist->GetBinContent(i, j);
            double center = hist->GetYaxis()->GetBinCenter(j);
            sum_y += value * center;
            totalEntries += value;
        }

        double average = (totalEntries > 0) ? sum_y / totalEntries : 0;
        graph->SetBinContent(i, average);
        const char* label = hist->GetXaxis()->GetBinLabel(i);
        graph->GetXaxis()->SetBinLabel(i, label);
    }

    graph->SetMarkerStyle(8);
    return graph;
}


In [19]:
%%cpp

void MicromegasDrawClusterInfo(TFile* qa_file_new) {
        std::cout << "MicromegasDrawClusterInfo Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }


    // Retrieve histograms
    auto h_cluster_count_ref = static_cast<TH1*>(qa_file_new->Get("h_MicromegasClusterQA_clustercount_ref"));
    auto h_cluster_count_found = static_cast<TH1*>(qa_file_new->Get("h_MicromegasClusterQA_clustercount_found"));
    auto h_cluster_multiplicity_raw = static_cast<TH2*>(qa_file_new->Get("h_MicromegasClusterQA_cluster_multiplicity"));
    auto h_cluster_size_raw = static_cast<TH2*>(qa_file_new->Get("h_MicromegasClusterQA_cluster_size"));
    auto h_cluster_charge_raw = static_cast<TH2*>(qa_file_new->Get("h_MicromegasClusterQA_cluster_charge"));

    // Check if histograms exist
    if (!h_cluster_count_ref || !h_cluster_count_found || !h_cluster_multiplicity_raw || !h_cluster_size_raw || !h_cluster_charge_raw) {
        std::cerr << "Error: One or more histograms could not be retrieved." << std::endl;
        return;
    }

    // Compute efficiency
    auto efficiency = static_cast<TH1*>(h_cluster_count_found->Clone("efficiency"));
    efficiency->Divide(h_cluster_count_ref);

    // Compute averages
    auto h_cluster_multiplicity = ClusterAverage(h_cluster_multiplicity_raw, "mult");
    auto h_cluster_size = ClusterAverage(h_cluster_size_raw, "size");
    auto h_cluster_charge = ClusterAverage(h_cluster_charge_raw, "charge");

    // Create canvas
    TCanvas *canvas = new TCanvas("ClusterQA", "Micromegas Cluster QA", 1000, 800);
    canvas->Divide(2, 2);

    // Draw Cluster Multiplicity
    canvas->cd(1);
    h_cluster_multiplicity->SetTitle("Cluster Multiplicity");
    h_cluster_multiplicity->GetXaxis()->SetTitle("Chamber");
    h_cluster_multiplicity->GetYaxis()->SetTitle("Multiplicity");
    h_cluster_multiplicity->SetMinimum(0);
    h_cluster_multiplicity->SetMaximum(10);
    h_cluster_multiplicity->DrawCopy("P");

    // Draw Cluster Size
    canvas->cd(2);
    h_cluster_size->SetTitle("Cluster Size");
    h_cluster_size->GetXaxis()->SetTitle("Chamber");
    h_cluster_size->GetYaxis()->SetTitle("Size");
    h_cluster_size->SetMinimum(0);
    h_cluster_size->SetMaximum(8);
    h_cluster_size->DrawCopy("P");

    // Draw Cluster Charge
    canvas->cd(3);
    h_cluster_charge->SetTitle("Cluster Charge");
    h_cluster_charge->GetXaxis()->SetTitle("Chamber");
    h_cluster_charge->GetYaxis()->SetTitle("Charge");
    h_cluster_charge->SetMinimum(0);
    h_cluster_charge->SetMaximum(1000);
    h_cluster_charge->DrawCopy("P");

    // Draw Efficiency Estimate
    canvas->cd(4);
    efficiency->SetMinimum(0);
    efficiency->SetMaximum(1);
    efficiency->SetTitle("Efficiency Estimate by Chamber");
    efficiency->GetXaxis()->SetTitle("Chamber");
    efficiency->GetYaxis()->SetTitle("Efficiency");
    efficiency->DrawCopy();

    // Update canvas
    canvas->Update();
}


In [20]:
%%cpp
MicromegasDrawClusterInfo(qa_file_new)

MicromegasDrawClusterInfo Beginning


Warning in <TFile::Append>: Replacing existing TH1: avg_mult (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: avg_size (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: avg_charge (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: ClusterQA


In [16]:
# Micromegas BCO Information

In [21]:
%%cpp

int MicromegasDrawBCOInfo(TFile* qa_file_new) {
    std::cout << "MicromegasDrawBCOInfo Beginning" << std::endl;
        // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return -1;
    }

    // Retrieve histograms
    auto h_waveform_bco_dropped = static_cast<TH1*>(qa_file_new->Get("h_MicromegasBCOQA_waveform_count_dropped_bco"));
    auto h_waveform_pool_dropped = static_cast<TH1*>(qa_file_new->Get("h_MicromegasBCOQA_waveform_count_dropped_pool"));
    auto h_waveform_total = static_cast<TH1*>(qa_file_new->Get("h_MicromegasBCOQA_waveform_count_total"));
    auto h_gl1_raw = static_cast<TH1*>(qa_file_new->Get("h_MicromegasBCOQA_packet_stat"));

    if (!h_waveform_bco_dropped || !h_waveform_pool_dropped || !h_waveform_total || !h_gl1_raw) {
        std::cerr << "Error: One or more histograms could not be retrieved." << std::endl;
        return -1;
    }

    // Create Drop Rate Histogram
    auto h_drop = new TH1F("h_drop", "Drop Rate", 3, 0, 3);
    h_drop->GetXaxis()->SetBinLabel(1, "5001");
    h_drop->GetXaxis()->SetBinLabel(2, "5002");
    h_drop->GetXaxis()->SetBinLabel(3, "all");
    h_drop->GetXaxis()->SetTitle("Packet");
    h_drop->GetYaxis()->SetTitle("Waveform Drop Rate");
    h_drop->SetTitle("Fraction of Dropped Waveforms by Packet");

    // Calculate drop rates
    h_drop->SetBinContent(1, double(h_waveform_bco_dropped->GetBinContent(1) + h_waveform_pool_dropped->GetBinContent(1)) / h_waveform_total->GetBinContent(1));
    h_drop->SetBinContent(2, double(h_waveform_bco_dropped->GetBinContent(2) + h_waveform_pool_dropped->GetBinContent(2)) / h_waveform_total->GetBinContent(2));
    h_drop->SetBinContent(3, double(h_waveform_bco_dropped->GetBinContent(1) + h_waveform_pool_dropped->GetBinContent(1) +
                                    h_waveform_bco_dropped->GetBinContent(2) + h_waveform_pool_dropped->GetBinContent(2)) /
                                    (h_waveform_total->GetBinContent(1) + h_waveform_total->GetBinContent(2)));

    // Create Match Rate Histogram
    auto h_gl1 = new TH1F("h_gl1", "Match Rate", 3, 0, 3);
    h_gl1->GetXaxis()->SetBinLabel(1, "5001");
    h_gl1->GetXaxis()->SetBinLabel(2, "5002");
    h_gl1->GetXaxis()->SetBinLabel(3, "all");
    h_gl1->GetXaxis()->SetTitle("Packet");
    h_gl1->GetYaxis()->SetTitle("Match Rate");
    h_gl1->SetTitle("Matching Tagger Rate by Packet");

    // Calculate match rates
    h_gl1->SetBinContent(1, double(h_gl1_raw->GetBinContent(2)) / h_gl1_raw->GetBinContent(1));
    h_gl1->SetBinContent(2, double(h_gl1_raw->GetBinContent(3)) / h_gl1_raw->GetBinContent(1));
    h_gl1->SetBinContent(3, double(h_gl1_raw->GetBinContent(4)) / h_gl1_raw->GetBinContent(1));

    // Create canvas
    TCanvas *canvas = new TCanvas("BCO_QA", "Micromegas BCO QA", 1000, 800);
    canvas->Divide(2, 1);

    // Draw Drop Rate Histogram
    canvas->cd(1);
    h_drop->SetMinimum(0);
    h_drop->SetMaximum(1.6);
    h_drop->SetFillColor(42);
    h_drop->SetFillStyle(3002);
    h_drop->Draw("HIST");

    // Add legend for drop rates
    TLegend* legend_drop = new TLegend(0.56, 0.6, 0.85, 0.84);
    legend_drop->SetHeader("Values", "C");
    legend_drop->SetTextSize(0.045);
    legend_drop->SetBorderSize(0);
    legend_drop->SetFillStyle(0);
    for (int i = 1; i <= h_drop->GetNbinsX(); ++i) {
        legend_drop->AddEntry((TObject*)0, Form("%s: %.4f", h_drop->GetXaxis()->GetBinLabel(i), h_drop->GetBinContent(i)), "");
    }
    legend_drop->Draw();

    // Draw Match Rate Histogram
    canvas->cd(2);
    h_gl1->SetMinimum(0);
    h_gl1->SetMaximum(1.6);
    h_gl1->SetFillColor(42);
    h_gl1->SetFillStyle(3002);
    h_gl1->Draw("HIST");

    // Add legend for match rates
    TLegend* legend_gl1 = new TLegend(0.65, 0.6, 0.85, 0.84);
    legend_gl1->SetHeader("Values", "C");
    legend_gl1->SetTextSize(0.045);
    legend_gl1->SetBorderSize(0);
    legend_gl1->SetFillStyle(0);
    for (int i = 1; i <= h_gl1->GetNbinsX(); ++i) {
        legend_gl1->AddEntry((TObject*)0, Form("%s: %.4f", h_gl1->GetXaxis()->GetBinLabel(i), h_gl1->GetBinContent(i)), "");
    }
    legend_gl1->Draw();

    // Update canvas
    canvas->Update();

    return 0;
}


In [22]:
%%cpp
MicromegasDrawBCOInfo(qa_file_new) 

MicromegasDrawBCOInfo Beginning
(int) 0


Warning in <TFile::Append>: Replacing existing TH1: h_drop (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_gl1 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: BCO_QA
